In [1]:
import customtkinter as ctk
import tkinter as tk
from tkinter import messagebox, filedialog
import os
import json
from collections import deque
import time
import threading

# --- INI ADALAH INTEGRASI PYGAME ---
# PENTING: Pygame digunakan untuk memutar audio
try:
    import pygame
    # Inisialisasi mixer pygame
    pygame.mixer.init()
    print("[INFO] Pygame Mixer berhasil diinisialisasi.")
    PYGAME_AVAILABLE = True
except ImportError:
    print("Peringatan: Pustaka 'pygame' tidak ditemukan. Fungsionalitas pemutaran audio mungkin tidak berfungsi.")
    pygame = None
    PYGAME_AVAILABLE = False
# -----------------------------------

# Set default appearance for customtkinter
ctk.set_appearance_mode("Dark")
ctk.set_default_color_theme("blue")

# --- 1. STRUKTUR DATA APLIKASI ---

# --- KONSTANTA FILE ---
LIBRARY_FILE = 'syfoplay_library_gui.json'
USERS_FILE = 'syfoplay_users_gui.json'

# 1. DATA MODELS / NODES
class Lagu:
    """Record node untuk lagu dengan file_audio yang mengacu ke path fisik."""
    def __init__(self, id_lagu, judul, artis, genre, album, durasi, file_audio="simulated_path.mp3"):
        self.id_lagu = id_lagu
        self.judul = judul
        self.artis = artis
        self.genre = genre
        self.album = album
        self.durasi = durasi
        # Path file audio, digunakan oleh pygame
        self.file_audio = file_audio
        self.next_lagu = None
        self.next_artis = None
        self.next_genre = None

class User:
    """Tipe data pengguna."""
    def __init__(self, email, username, password):
        self.email = email
        self.username = username
        self.password = password
        self.playlists = {}

    def to_dict(self):
        """Mengubah objek User menjadi dictionary untuk disimpan di JSON."""
        playlists_serial = {}
        # PlaylistLinkedList tidak bisa langsung diserialisasi, jadi simpan ID lagunya
        for name, playlist_ll in self.playlists.items():
            playlists_serial[name] = playlist_ll.to_list_id()
        return {
            "email": self.email,
            "username": self.username,
            "password": self.password,
            "playlists": playlists_serial
        }

# 2. PLAYLIST (Singly Linked List)
class NodePlaylist:
    def __init__(self, lagu):
        self.lagu = lagu
        self.next = None

class PlaylistLinkedList:
    def __init__(self):
        self.head = None
        self.tail = None

    def tambah_lagu(self, lagu):
        """Menambahkan lagu ke akhir linked list."""
        if self.find_lagu_by_id(lagu.id_lagu):
            return False # Lagu sudah ada
            
        new_node = NodePlaylist(lagu)
        if not self.head:
            self.head = new_node
            self.tail = new_node
        else:
            self.tail.next = new_node
            self.tail = new_node
        return True

    def find_lagu_by_id(self, id_lagu):
        """Mencari lagu di playlist berdasarkan ID."""
        curr = self.head
        while curr:
            if curr.lagu.id_lagu == id_lagu:
                return curr.lagu
            curr = curr.next
        return None

    def hapus_lagu_berdasarkan_id(self, id_lagu):
        """Menghapus lagu berdasarkan ID dari playlist."""
        if not self.head: return False
        if self.head.lagu.id_lagu == id_lagu:
            self.head = self.head.next
            if not self.head: self.tail = None
            return True
        curr = self.head
        while curr.next:
            if curr.next.lagu.id_lagu == id_lagu:
                if curr.next == self.tail: self.tail = curr
                curr.next = curr.next.next
                return True
            curr = curr.next
        return False
    
    def to_list_id(self):
        ids = []
        curr = self.head
        while curr:
            ids.append(curr.lagu.id_lagu)
            curr = curr.next
        return ids
    
    def get_lagu_list(self):
        """Mengembalikan list objek Lagu untuk tampilan GUI."""
        lagu_list = []
        curr = self.head
        while curr:
            lagu_list.append(curr.lagu)
            curr = curr.next
        return lagu_list

# 3. LIBRARY (Singly Linked List)
class LibraryMultiLinkedList:
    def __init__(self):
        self.head = None
        self.lagu_counter = 1

    def _generate_id(self):
        id_baru = f"L{self.lagu_counter:03}"
        self.lagu_counter += 1
        return id_baru

    def tambah_lagu(self, judul, artis, genre, album, durasi, file_audio): 
        """Menambahkan lagu baru ke head dari Linked List."""
        id_baru = self._generate_id()
        lagu_baru = Lagu(id_baru, judul, artis, genre, album, durasi, file_audio)
        lagu_baru.next_lagu = self.head
        self.head = lagu_baru
        return lagu_baru

    def get_all_lagu(self):
        """Mengembalikan list semua objek Lagu."""
        lagu_list = []
        curr = self.head
        while curr:
            lagu_list.append(curr)
            curr = curr.next_lagu
        # Sort by ID for consistent display across sessions
        lagu_list.sort(key=lambda l: l.id_lagu) 
        return lagu_list

    def cari_lagu(self, kunci, nilai):
        """Melakukan pencarian berdasarkan kriteria."""
        hasil = []
        curr = self.head
        nilai_lower = nilai.lower()
        while curr:
            if kunci == "judul" and nilai_lower in curr.judul.lower():
                hasil.append(curr)
            elif kunci == "artis" and nilai_lower in curr.artis.lower():
                hasil.append(curr)
            elif kunci == "genre" and nilai_lower in curr.genre.lower():
                hasil.append(curr)
            elif kunci == "album" and nilai_lower in curr.album.lower():
                hasil.append(curr)
            curr = curr.next_lagu
        return hasil

    def cari_lagu_berdasarkan_id(self, id_lagu):
        """Mencari lagu berdasarkan ID."""
        curr = self.head
        while curr:
            if curr.id_lagu == id_lagu:
                return curr
            curr = curr.next_lagu
        return None

    def hapus_lagu_dan_update_playlist(self, id_lagu, all_users):
        """Hapus lagu dari library dan dari semua playlist user."""
        if not self.head: return False
        lagu_dihapus = None
        if self.head.id_lagu == id_lagu:
            lagu_dihapus = self.head
            self.head = self.head.next_lagu
        else:
            curr = self.head
            while curr.next_lagu:
                if curr.next_lagu.id_lagu == id_lagu:
                    lagu_dihapus = curr.next_lagu
                    curr.next_lagu = curr.next_lagu.next_lagu
                    break
                curr = curr.next_lagu
        if not lagu_dihapus: return False
        
        # Hapus dari semua playlist user
        for username, user in all_users.items():
            for nama_playlist, playlist_ll in user.playlists.items():
                playlist_ll.hapus_lagu_berdasarkan_id(id_lagu)
        return True

    def find_similar_lagu_genre(self, genre_target, exclude_id=None, limit=5):
        """Mencari lagu lain yang memiliki genre yang sama."""
        hasil = []
        curr = self.head
        while curr and len(hasil) < limit:
            if curr.genre.lower() == genre_target.lower() and curr.id_lagu != exclude_id:
                hasil.append(curr)
            curr = curr.next_lagu
        return hasil


# 4. Playback queue (deque) dan history (stack)
class PlaybackQueue:
    def __init__(self):
        self._items = deque()
    def enqueue(self, lagu): self._items.append(lagu)
    def dequeue(self): return self._items.popleft() if self._items else None
    def is_empty(self): return len(self._items) == 0
    def to_list(self): return list(self._items)
    # Tambahkan metode __len__ untuk PlaybackQueue juga
    def __len__(self):
        return len(self._items)


class PlaybackHistory:
    def __init__(self):
        self._stack = []
    def push(self, lagu): self._stack.append(lagu)
    def pop(self): return self._stack.pop() if self._stack else None
    def is_empty(self): return len(self._stack) == 0
    def to_list(self): return list(self._stack)
    
    # METODE PERBAIKAN: Memungkinkan penggunaan len()
    def __len__(self):
        return len(self._stack)


# 5. APLIKASI UTAMA (BACKEND)
class SyfoplayAppLogic:
    def __init__(self):
        self.LIBRARY_FILE = LIBRARY_FILE
        self.USERS_FILE = USERS_FILE
        self.library = LibraryMultiLinkedList()
        self.users = {}
        self.admin_id = "admin"
        self.admin_pass = "admin123"
        self.current_user = None # Objek User yang sedang login
        
        # Playback Control
        self.current_playing = None # Objek Lagu yang sedang diputar
        self.playback_queue = PlaybackQueue()
        self.history = PlaybackHistory()
        self.playback_status = "STOPPED" # "PLAYING", "PAUSED", "STOPPED"
        self.playback_volume = 0.5 # Default volume

        self._load_data()
        
    # --- JSON PERSISTENCE METHODS ---
    def _load_data(self):
        # 1. Load Library
        try:
            with open(self.LIBRARY_FILE, 'r') as f:
                data_lib = json.load(f)
            
            self.library.lagu_counter = data_lib.get("lagu_counter", 1)
            
            head_node = None
            # Memuat lagu dari list, harus dari yang terakhir ditambahkan agar urutan LL benar
            for item in reversed(data_lib.get("library", [])): 
                lagu = Lagu(**item)
                lagu.next_lagu = head_node
                head_node = lagu
            self.library.head = head_node
        except (FileNotFoundError, json.JSONDecodeError):
            print("[INFO] File Library tidak ditemukan/rusak. Menggunakan library kosong.")

        # 2. Load Users
        try:
            with open(self.USERS_FILE, 'r') as f:
                data_users = json.load(f)

            for username, user_data in data_users.items():
                user = User(user_data["email"], user_data["username"], user_data["password"])
                
                for name, list_id_lagu in user_data.get("playlists", {}).items():
                    playlist_ll = PlaylistLinkedList()
                    for id_lagu in list_id_lagu:
                        lagu_ref = self.library.cari_lagu_berdasarkan_id(id_lagu)
                        if lagu_ref:
                            playlist_ll.tambah_lagu(lagu_ref)
                    user.playlists[name] = playlist_ll
                
                self.users[username] = user
        except (FileNotFoundError, json.JSONDecodeError):
            print("[INFO] File Users tidak ditemukan/rusak. Menggunakan user kosong.")
            
    def _save_library(self):
        data_lib = {
            "lagu_counter": self.library.lagu_counter,
            "library": []
        }
        curr = self.library.head
        while curr:
            lagu_dict = vars(curr).copy()
            # Hapus properti linked list sebelum menyimpan
            lagu_dict.pop('next_lagu', None)
            lagu_dict.pop('next_artis', None)
            lagu_dict.pop('next_genre', None)
            data_lib["library"].append(lagu_dict)
            curr = curr.next_lagu
        
        # Simpan dalam urutan ID yang konsisten
        data_lib["library"].sort(key=lambda x: x['id_lagu']) 
        
        try:
            with open(self.LIBRARY_FILE, 'w') as f:
                json.dump(data_lib, f, indent=2)
        except Exception as e:
            print(f"[ERROR] Gagal menyimpan library: {e}")

    def _save_users(self):
        data_users = {}
        for username, user in self.users.items():
            data_users[username] = user.to_dict()

        try:
            with open(self.USERS_FILE, 'w') as f:
                json.dump(data_users, f, indent=2)
        except Exception as e:
            print(f"[ERROR] Gagal menyimpan users: {e}")

    # --- PYGAME & PLAYBACK LOGIC ---

    def play_lagu(self, lagu):
        """Memulai pemutaran lagu baru."""
        if not PYGAME_AVAILABLE:
            self.current_playing = lagu
            self.playback_status = "PLAYING (Simulasi)"
            return
            
        if self.current_playing and self.current_playing != lagu:
            self.history.push(self.current_playing)
            
        self.current_playing = lagu
        
        # Memuat file audio dan memutar
        try:
            # Pengecekan jika file_audio adalah path yang valid
            if not os.path.exists(lagu.file_audio):
                # Ganti dengan path ke file dummy jika path tidak valid dan bukan path simulasi
                if lagu.file_audio != "simulated_path.mp3":
                    raise FileNotFoundError(f"File audio tidak ditemukan: {lagu.file_audio}")

            pygame.mixer.music.load(lagu.file_audio)
            pygame.mixer.music.set_volume(self.playback_volume)
            pygame.mixer.music.play()
            self.playback_status = "PLAYING"
            
            # Memulai thread untuk memantau status lagu selesai
            self._start_monitor_thread()
            
        except FileNotFoundError as e:
            messagebox.showerror("Error Audio", str(e))
            self.current_playing = None
            self.playback_status = "STOPPED"
        except pygame.error as e:
            # Ini mungkin terjadi jika file dummy path tidak valid atau format tidak didukung
            messagebox.showerror("Error Pygame", f"Gagal memutar audio: {e}")
            self.current_playing = None
            self.playback_status = "STOPPED"

    def pause_lagu(self):
        """Menjeda pemutaran."""
        if not PYGAME_AVAILABLE or self.playback_status != "PLAYING": return
        pygame.mixer.music.pause()
        self.playback_status = "PAUSED"

    def unpause_lagu(self):
        """Melanjutkan pemutaran dari jeda."""
        if not PYGAME_AVAILABLE or self.playback_status != "PAUSED": return
        pygame.mixer.music.unpause()
        self.playback_status = "PLAYING"

    def stop_lagu(self):
        """Menghentikan pemutaran sepenuhnya."""
        if not PYGAME_AVAILABLE:
            self.current_playing = None
            self.playback_status = "STOPPED"
            return
            
        pygame.mixer.music.stop()
        self.playback_status = "STOPPED"
        self.current_playing = None

    def next_queue(self):
        """Memutar lagu berikutnya di queue (FIFO)."""
        lagu_next = self.playback_queue.dequeue()
        if lagu_next:
            self.play_lagu(lagu_next)
            return True
        else:
            self.stop_lagu()
            return False

    def prev_history(self):
        """Memutar lagu sebelumnya dari history (LIFO)."""
        lagu_prev = self.history.pop()
        if lagu_prev:
            if self.current_playing:
                # Simpan lagu yang sedang diputar ke depan queue saat mundur
                self.playback_queue._items.appendleft(self.current_playing) 
            self.play_lagu(lagu_prev)
            return True
        else:
            return False

    def set_volume(self, value):
        """Mengatur volume dan menyimpan nilai di logic."""
        self.playback_volume = float(value)
        if PYGAME_AVAILABLE:
            pygame.mixer.music.set_volume(self.playback_volume)


    def _monitor_playback_completion(self):
        """Thread yang memantau kapan lagu selesai diputar."""
        if not PYGAME_AVAILABLE: return

        # Loop pemantauan, akan berhenti saat lagu selesai atau status berubah
        while self.playback_status == "PLAYING" and self.current_playing:
            # Cek jika lagu sudah berhenti dan posisinya lebih dari 0 (berarti selesai)
            if not pygame.mixer.music.get_busy() and pygame.mixer.music.get_pos() > 0:
                break
            time.sleep(0.5)

        # Hanya jalankan next_queue jika lagu memang selesai secara normal
        if self.playback_status == "PLAYING" and self.current_playing:
            self.next_queue()
            # Pembaruan GUI harus dilakukan di main thread
            if hasattr(self, 'gui_controller'):
                # Gunakan after untuk memanggil di main thread
                self.gui_controller.after(0, self.gui_controller.playback_bar.update_playback_info)
    
    def _start_monitor_thread(self):
        """Memulai thread pemantauan jika belum berjalan."""
        if PYGAME_AVAILABLE:
            # Pastikan hanya satu monitor thread per pemutaran baru
            monitor_thread = threading.Thread(target=self._monitor_playback_completion, daemon=True)
            monitor_thread.start()


# --- 2. IMPLEMENTASI GUI DENGAN CUSTOMTKINTER ---

# --- PLAYBACK BAR DI BAWAH (Muncul setelah login) ---
class PlaybackBar(ctk.CTkFrame):
    def __init__(self, master, logic):
        super().__init__(master, height=60, fg_color="#181818")
        self.logic = logic
        
        self.grid_columnconfigure(0, weight=1) # Info Lagu
        self.grid_columnconfigure(1, weight=1) # Kontrol Playback
        self.grid_columnconfigure(2, weight=1) # Volume
        
        # 1. Info Lagu (Kiri)
        self.song_info_frame = ctk.CTkFrame(self, fg_color="transparent")
        self.song_info_frame.grid(row=0, column=0, padx=10, pady=5, sticky="w")
        self.title_label = ctk.CTkLabel(self.song_info_frame, text="[Tidak Ada Lagu]", font=ctk.CTkFont(weight="bold"))
        self.title_label.grid(row=0, column=0, sticky="w")
        self.artist_label = ctk.CTkLabel(self.song_info_frame, text="Oleh: -")
        self.artist_label.grid(row=1, column=0, sticky="w")
        
        # 2. Kontrol Playback (Tengah)
        self.control_frame = ctk.CTkFrame(self, fg_color="transparent")
        self.control_frame.grid(row=0, column=1, padx=10, pady=5)
        
        self.prev_button = ctk.CTkButton(self.control_frame, text="⏮️", width=40, command=self.prev_song)
        self.prev_button.pack(side="left", padx=5)
        self.play_pause_button = ctk.CTkButton(self.control_frame, text="▶️", width=40, command=self.toggle_play_pause)
        self.play_pause_button.pack(side="left", padx=5)
        self.next_button = ctk.CTkButton(self.control_frame, text="⏭️", width=40, command=self.next_song)
        self.next_button.pack(side="left", padx=5)
        self.stop_button = ctk.CTkButton(self.control_frame, text="■", width=40, command=self.stop_playback)
        self.stop_button.pack(side="left", padx=5)
        
        # 3. Volume Control (Kanan)
        self.volume_frame = ctk.CTkFrame(self, fg_color="transparent")
        self.volume_frame.grid(row=0, column=2, padx=10, pady=5, sticky="e")
        ctk.CTkLabel(self.volume_frame, text="Volume:").pack(side="left", padx=5)
        self.volume_slider = ctk.CTkSlider(self.volume_frame, from_=0, to=1, command=self.set_volume, width=100)
        self.volume_slider.set(self.logic.playback_volume)
        self.volume_slider.pack(side="left", padx=5)

        self.update_playback_info() # Panggil untuk inisialisasi awal

    def toggle_play_pause(self):
        if not self.logic.current_playing:
            # Coba putar lagu berikutnya di queue jika ada (untuk Admin/User yang sudah mengisi queue)
            if not self.logic.next_queue():
                messagebox.showwarning("Playback", "Tidak ada lagu yang dipilih atau di-queue.")
        elif self.logic.playback_status == "PLAYING":
            self.logic.pause_lagu()
        elif self.logic.playback_status == "PAUSED":
            self.logic.unpause_lagu()
        self.update_playback_info()
        
    def stop_playback(self):
        self.logic.stop_lagu()
        self.update_playback_info()


    def set_volume(self, value):
        self.logic.set_volume(value)

    def next_song(self):
        self.logic.next_queue()
        self.update_playback_info()

    def prev_song(self):
        self.logic.prev_history()
        self.update_playback_info()

    def update_playback_info(self):
        """Memperbarui label info dan tombol berdasarkan status logika."""
        lagu = self.logic.current_playing
        
        if lagu:
            status = f" ({self.logic.playback_status})"
            self.title_label.configure(text=lagu.judul)
            self.artist_label.configure(text=f"Oleh: {lagu.artis}")
        else:
            status = ""
            self.title_label.configure(text="[Tidak Ada Lagu]")
            self.artist_label.configure(text="Oleh: -")

        # Update Tombol Play/Pause
        if self.logic.playback_status == "PLAYING":
            self.play_pause_button.configure(text="⏸️")
        else:
            self.play_pause_button.configure(text="▶️")
            
        # Update warna tombol Prev/Next (simulasi disable)
        self.prev_button.configure(state=tk.NORMAL if len(self.logic.history) > 0 else tk.DISABLED)
        self.next_button.configure(state=tk.NORMAL if not self.logic.playback_queue.is_empty() else tk.DISABLED)


# --- STRUKTUR APLIKASI UTAMA (MAIN APP) ---
class SyfoplayGUI(ctk.CTk):
    def __init__(self, logic):
        super().__init__()
        self.logic = logic
        self.logic.gui_controller = self # Berikan referensi ke logic untuk update thread
        
        self.title("Syfoplay Aplikasi Musik")
        self.geometry("800x600")
        self.min_width = 800
        self.min_height = 600
        self.grid_rowconfigure(0, weight=1) # Konten Utama
        self.grid_rowconfigure(1, weight=0) # Playback Bar
        self.grid_columnconfigure(0, weight=1)

        # Container Frame (Menampung semua halaman)
        self.container = ctk.CTkFrame(self)
        self.container.grid(row=0, column=0, sticky="nsew")
        self.container.grid_rowconfigure(0, weight=1)
        self.container.grid_columnconfigure(0, weight=1)

        # Playback Bar (Diletakkan di grid row 1, visibility dikontrol)
        self.playback_bar = PlaybackBar(self, self.logic)
        self.playback_bar.grid(row=1, column=0, sticky="ew")
        self.playback_bar.grid_remove() # Sembunyikan secara default

        self.frames = {}
        self.create_frames()
        self.show_frame("LoginPage") 
        
        self.protocol("WM_DELETE_WINDOW", self.on_closing)

    def on_closing(self):
        """Memastikan data tersimpan saat aplikasi ditutup."""
        try:
            self.logic.stop_lagu()
            self.logic._save_library()
            self.logic._save_users()
            print("Data tersimpan sebelum keluar.")
        except Exception as e:
            print(f"Gagal menyimpan data saat keluar: {e}")
        self.destroy()

    def create_frames(self):
        # Inisialisasi semua frame/halaman
        for F in (LoginPage, UserSignupPage, AdminDashboardPage, UserDashboardPage, UserPlaylistMenuPage, AdminAddUpdateSongPage, UserArtistListPage, UserSearchPage):
            page_name = F.__name__
            frame = F(parent=self.container, controller=self, logic=self.logic)
            self.frames[page_name] = frame
            frame.grid(row=0, column=0, sticky="nsew")

    def show_frame(self, page_name, *args, **kwargs):
        """Menampilkan halaman yang diminta."""
        frame = self.frames[page_name]
        
        # Panggil metode refresh (jika ada) sebelum ditampilkan
        if hasattr(frame, 'refresh_content'):
            frame.refresh_content(*args, **kwargs)

        frame.tkraise()
        self.playback_bar.update_playback_info() 
        
    def toggle_playback_bar(self, show):
        """Menampilkan atau menyembunyikan Playback Bar."""
        if show:
            self.playback_bar.grid()
        else:
            self.playback_bar.grid_remove()
            self.logic.stop_lagu() # Berhenti memutar saat logout/keluar

    def get_frame(self, page_name):
        return self.frames[page_name]


# --- 3. HALAMAN-HALAMAN APLIKASI ---

# Halaman Login/Menu Utama (Admin Login)
class LoginPage(ctk.CTkFrame):
    def __init__(self, parent, controller, logic):
        super().__init__(parent, fg_color="#204060") 
        self.controller = controller
        self.logic = logic
        
        self.grid_columnconfigure(0, weight=1)
        self.grid_rowconfigure(0, weight=1)

        # Frame Login Admin (Tengah)
        login_frame = ctk.CTkFrame(self, width=350, height=450, corner_radius=10)
        login_frame.grid(row=0, column=0, padx=20, pady=20, sticky="nsew")
        login_frame.pack_propagate(False) 
        
        ctk.CTkLabel(login_frame, text="SyfoPlay", font=ctk.CTkFont(size=36, weight="bold")).pack(pady=(40, 10))
        ctk.CTkLabel(login_frame, text="LOGIN ADMIN", font=ctk.CTkFont(size=18, weight="bold")).pack(pady=10)

        ctk.CTkLabel(login_frame, text="ID Admin").pack(pady=(10, 0))
        self.id_entry = ctk.CTkEntry(login_frame, width=250)
        self.id_entry.pack(pady=(0, 10))

        ctk.CTkLabel(login_frame, text="Password").pack(pady=(10, 0))
        self.pass_entry = ctk.CTkEntry(login_frame, width=250, show="*")
        self.pass_entry.pack(pady=(0, 20))

        ctk.CTkButton(login_frame, text="Login Admin", command=self.login_admin).pack(pady=10)
        ctk.CTkButton(login_frame, text="User (Login/Sign Up)", command=lambda: controller.show_frame("UserSignupPage")).pack(pady=5)
        
        # PERBAIKAN: Tombol keluar aplikasi
        ctk.CTkButton(login_frame, text="❌ Keluar Aplikasi", command=controller.destroy, fg_color="red", hover_color="#800000").pack(pady=(20, 10))

    def login_admin(self):
        admin_id = self.id_entry.get().strip()
        admin_pass = self.pass_entry.get().strip()
        
        if admin_id == self.logic.admin_id and admin_pass == self.logic.admin_pass:
            messagebox.showinfo("Login Berhasil", "Selamat datang, Admin!")
            self.id_entry.delete(0, tk.END)
            self.pass_entry.delete(0, tk.END)
            self.controller.toggle_playback_bar(show=True) # Tampilkan Playback Bar
            self.controller.show_frame("AdminDashboardPage")
        else:
            messagebox.showerror("Login Gagal", "ID Admin atau Password salah.")


# Halaman Sign Up / Login User
class UserSignupPage(ctk.CTkFrame):
    def __init__(self, parent, controller, logic):
        super().__init__(parent)
        self.controller = controller
        self.logic = logic
        self.current_mode = tk.StringVar(value="signup")

        self.grid_columnconfigure(0, weight=1)
        self.grid_rowconfigure(0, weight=1)
        
        self.main_frame = ctk.CTkFrame(self, width=400, height=500, corner_radius=10)
        self.main_frame.grid(row=0, column=0, padx=20, pady=20, sticky="nsew")
        self.main_frame.pack_propagate(False) 

        # Judul
        ctk.CTkLabel(self.main_frame, text="AKSES PENGGUNA", font=ctk.CTkFont(size=24, weight="bold")).pack(pady=(20, 10))

        # Switch Button (Sign Up / Login)
        self.switch_frame = ctk.CTkFrame(self.main_frame, fg_color="transparent")
        self.switch_frame.pack(pady=10)
        self.login_switch = ctk.CTkButton(self.switch_frame, text="Login", command=lambda: self.switch_mode("login"))
        self.login_switch.grid(row=0, column=0, padx=5)
        self.signup_switch = ctk.CTkButton(self.switch_frame, text="Sign Up", command=lambda: self.switch_mode("signup"))
        self.signup_switch.grid(row=0, column=1, padx=5)

        # Area Konten
        self.content_frame = ctk.CTkFrame(self.main_frame, fg_color="transparent")
        self.content_frame.pack(padx=20, pady=10, fill="x")
        
        # Tombol Kembali
        ctk.CTkButton(self.main_frame, text="< KEMBALI", command=lambda: controller.show_frame("LoginPage")).pack(pady=10)
        
        self.switch_mode("signup") # Tampilkan Sign Up pertama kali

    def switch_mode(self, mode):
        """Mengganti tampilan antara Sign Up dan Login."""
        if self.current_mode.get() == mode:
            return

        for widget in self.content_frame.winfo_children():
            widget.destroy()

        self.current_mode.set(mode)
        
        # Atur warna tombol switch
        active_color = self.login_switch.cget("fg_color")
        inactive_color = "gray"
        
        self.login_switch.configure(fg_color=active_color if mode == "login" else inactive_color)
        self.signup_switch.configure(fg_color=active_color if mode == "signup" else inactive_color)
        
        if mode == "signup":
            self._create_signup_form()
        elif mode == "login":
            self._create_login_form()

    def _create_signup_form(self):
        
        ctk.CTkLabel(self.content_frame, text="BUAT AKUN BARU", font=ctk.CTkFont(weight="bold")).pack(pady=10)

        self.signup_email = ctk.CTkEntry(self.content_frame, placeholder_text="Email", width=250)
        self.signup_email.pack(pady=5)
        self.signup_username = ctk.CTkEntry(self.content_frame, placeholder_text="Username", width=250)
        self.signup_username.pack(pady=5)
        self.signup_password = ctk.CTkEntry(self.content_frame, placeholder_text="Password", show="*", width=250)
        self.signup_password.pack(pady=5)
        self.signup_verify_password = ctk.CTkEntry(self.content_frame, placeholder_text="Verifikasi Password", show="*", width=250)
        self.signup_verify_password.pack(pady=5)
        
        ctk.CTkButton(self.content_frame, text="Daftar", command=self.perform_signup).pack(pady=15)

    def _create_login_form(self):
        ctk.CTkLabel(self.content_frame, text="LOGIN PENGGUNA", font=ctk.CTkFont(weight="bold")).pack(pady=10)
        
        self.login_username = ctk.CTkEntry(self.content_frame, placeholder_text="Username", width=250)
        self.login_username.pack(pady=5)
        self.login_password = ctk.CTkEntry(self.content_frame, placeholder_text="Password", show="*", width=250)
        self.login_password.pack(pady=5)
        
        ctk.CTkButton(self.content_frame, text="Masuk", command=self.perform_login).pack(pady=15)

    def perform_signup(self):
        email = self.signup_email.get().strip()
        username = self.signup_username.get().strip()
        password = self.signup_password.get().strip()
        verify_password = self.signup_verify_password.get().strip()

        if not (email and username and password and verify_password):
            messagebox.showerror("Sign Up Gagal", "Semua field harus diisi.")
            return
        
        if password != verify_password:
            messagebox.showerror("Sign Up Gagal", "Verifikasi Password tidak sesuai.")
            return

        if username in self.logic.users:
            messagebox.showerror("Sign Up Gagal", "Username sudah terdaftar!")
            return

        # Buat akun baru dan simpan
        new_user = User(email, username, password)
        self.logic.users[username] = new_user
        self.logic._save_users()
        
        messagebox.showinfo("Sign Up Berhasil", f"Akun '{username}' berhasil dibuat. Silakan Login.")
        self.switch_mode("login") # Langsung pindah ke Login
        self.login_username.delete(0, tk.END)
        self.login_password.delete(0, tk.END)
        self.login_username.insert(0, username) # Isi otomatis username

    def perform_login(self):
        username = self.login_username.get().strip()
        password = self.login_password.get().strip()
        
        if not (username and password):
            messagebox.showerror("Login Gagal", "Username dan Password harus diisi.")
            return
        
        user = self.logic.users.get(username)
        if user and user.password == password:
            self.logic.current_user = user
            messagebox.showinfo("Login Berhasil", f"Selamat datang, {username}!")
            self.login_username.delete(0, tk.END)
            self.login_password.delete(0, tk.END)
            self.controller.toggle_playback_bar(show=True) # Tampilkan Playback Bar
            self.controller.show_frame("UserDashboardPage")
        else:
            messagebox.showerror("Login Gagal", "Username atau Password salah/belum terdaftar.")


# Dashboard Admin
class AdminDashboardPage(ctk.CTkFrame):
    def __init__(self, parent, controller, logic):
        super().__init__(parent)
        self.controller = controller
        self.logic = logic
        self.selected_song_id = None
        self.selected_row = None
        
        self.grid_rowconfigure(1, weight=0) # Kontrol
        self.grid_rowconfigure(2, weight=1) # List lagu
        self.grid_columnconfigure(0, weight=1)

        # Header Admin
        header_frame = ctk.CTkFrame(self, fg_color="#336699")
        header_frame.grid(row=0, column=0, sticky="ew", padx=0, pady=0)
        ctk.CTkLabel(header_frame, text="Hai Admin, selamat bekerja!", font=ctk.CTkFont(size=20, weight="bold")).pack(pady=10, padx=20, anchor="w")

        # Frame Kontrol (Search dan Aksi)
        control_frame = ctk.CTkFrame(self, fg_color="transparent")
        control_frame.grid(row=1, column=0, sticky="ew", padx=10, pady=10)
        control_frame.grid_columnconfigure(0, weight=3) # Search Input
        control_frame.grid_columnconfigure(1, weight=1) # Aksi Tombol

        # Search Frame
        search_frame = ctk.CTkFrame(control_frame)
        search_frame.grid(row=0, column=0, sticky="ew", padx=(0, 10))
        search_frame.grid_columnconfigure(1, weight=1)

        self.search_entry = ctk.CTkEntry(search_frame, placeholder_text="Cari berdasarkan Judul, Artis, Genre, atau Album...", width=300)
        self.search_entry.grid(row=0, column=0, sticky="ew", padx=(10, 5), pady=10)
        self.search_option = ctk.CTkOptionMenu(search_frame, values=["judul", "artis", "genre", "album"], width=100)
        self.search_option.grid(row=0, column=1, padx=(5, 10), pady=10)
        ctk.CTkButton(search_frame, text="CARI", command=self.cari_lagu, width=80).grid(row=0, column=2, padx=(0, 10), pady=10)

        # Action Frame (Kanan)
        action_frame = ctk.CTkFrame(control_frame)
        action_frame.grid(row=0, column=1, sticky="e")
        ctk.CTkButton(action_frame, text="➕ Tambah Lagu", command=lambda: controller.show_frame("AdminAddUpdateSongPage", mode="tambah")).pack(side="left", padx=5)
        self.update_button = ctk.CTkButton(action_frame, text="✏️ Update Lagu", command=self.open_update_song, state=tk.DISABLED)
        self.update_button.pack(side="left", padx=5)
        self.delete_button = ctk.CTkButton(action_frame, text="❌ Hapus Lagu", command=self.hapus_lagu, state=tk.DISABLED, fg_color="red", hover_color="#800000")
        self.delete_button.pack(side="left", padx=5)
        ctk.CTkButton(action_frame, text="🚪 Keluar", command=self.logout).pack(side="left", padx=5)

        # List Lagu (Scrollable)
        self.list_scroll_frame = ctk.CTkScrollableFrame(self)
        self.list_scroll_frame.grid(row=2, column=0, sticky="nsew", padx=10, pady=10)
        

        # List Frame untuk konten lagu di dalam scrollable frame
        self.list_frame = ctk.CTkFrame(self.list_scroll_frame, fg_color="transparent")
        self.list_frame.pack(fill="x", expand=True)
        # Konfigurasi kolom untuk list lagu (Header)
        self.list_frame.grid_columnconfigure(0, weight=0) # No
        self.list_frame.grid_columnconfigure(1, weight=0) # ID
        self.list_frame.grid_columnconfigure(2, weight=3) # Judul
        self.list_frame.grid_columnconfigure(3, weight=2) # Artis
        self.list_frame.grid_columnconfigure(4, weight=1) # Genre
        self.list_frame.grid_columnconfigure(5, weight=1) # Album
        self.list_frame.grid_columnconfigure(6, weight=0) # Durasi
        self.list_frame.grid_columnconfigure(7, weight=0) # Play Button

        self.refresh_content(reset_search=True)

    def refresh_content(self, search_results=None, reset_search=False):
        """Memuat ulang daftar lagu di tabel."""
        self.selected_song_id = None
        if self.selected_row:
             self.selected_row.configure(fg_color="transparent")
             self.selected_row = None

        self.update_button.configure(state=tk.DISABLED)
        self.delete_button.configure(state=tk.DISABLED)
        
        if reset_search:
            self.search_entry.delete(0, tk.END)

        # Hapus konten lama
        for widget in self.list_frame.winfo_children():
            widget.destroy()

        # Header Tabel
        headers = ["No", "ID", "Judul", "Artis", "Genre", "Album", "Durasi", "Aksi"]
        for i, text in enumerate(headers):
            ctk.CTkLabel(self.list_frame, text=text, font=ctk.CTkFont(weight="bold"), fg_color="#444444", height=30).grid(row=0, column=i, sticky="ew", padx=(1 if i > 0 else 0), pady=(0, 2))

        # Data Lagu
        lagu_list = search_results if search_results is not None else self.logic.library.get_all_lagu()
        
        for i, lagu in enumerate(lagu_list):
            row = i + 1
            
            # Frame untuk baris, agar bisa diberi warna latar belakang saat dipilih
            row_frame = ctk.CTkFrame(self.list_frame, fg_color="transparent", border_width=0)
            row_frame.grid(row=row, column=0, columnspan=8, sticky="ew", pady=(0, 1))

            # FIX: Tentukan bobot kolom di dalam row_frame agar align dengan header (Perbaikan Tampilan Tabel)
            row_frame.grid_columnconfigure(0, weight=0) # No
            row_frame.grid_columnconfigure(1, weight=0) # ID
            row_frame.grid_columnconfigure(2, weight=3) # Judul
            row_frame.grid_columnconfigure(3, weight=2) # Artis
            row_frame.grid_columnconfigure(4, weight=1) # Genre
            row_frame.grid_columnconfigure(5, weight=1) # Album
            row_frame.grid_columnconfigure(6, weight=0) # Durasi
            row_frame.grid_columnconfigure(7, weight=0) # Play Button (Aksi)

            def select_row_handler(event, song_id, frame):
                if self.selected_row:
                    self.selected_row.configure(fg_color="transparent")
                
                if self.selected_song_id == song_id:
                    self.selected_song_id = None
                    self.selected_row = None
                    self.update_button.configure(state=tk.DISABLED)
                    self.delete_button.configure(state=tk.DISABLED)
                else:
                    self.selected_song_id = song_id
                    self.selected_row = frame
                    self.selected_row.configure(fg_color="#334466")
                    self.update_button.configure(state=tk.NORMAL)
                    self.delete_button.configure(state=tk.NORMAL)

            row_frame.bind("<Button-1>", lambda event, sid=lagu.id_lagu, frame=row_frame: select_row_handler(event, sid, frame))

            # Kolom Data
            data_columns = [
                (f"{i+1}.", 0, tk.W),
                (lagu.id_lagu, 1, tk.W),
                (lagu.judul, 2, tk.W),
                (lagu.artis, 3, tk.W),
                (lagu.genre, 4, tk.W),
                (lagu.album, 5, tk.W),
                (lagu.durasi, 6, tk.W)
            ]
            
            for text, col, anchor in data_columns:
                label = ctk.CTkLabel(row_frame, text=text, anchor=anchor, padx=5, pady=3)
                label.grid(row=0, column=col, sticky="ew") # Menggunakan sticky="ew" agar label mengisi kolom
                label.bind("<Button-1>", lambda event, sid=lagu.id_lagu, frame=row_frame: select_row_handler(event, sid, frame))

            # Tombol Play
            ctk.CTkButton(row_frame, text="▶️", width=30, command=lambda l=lagu: self.play_song_admin(l)).grid(row=0, column=7, padx=5, pady=3, sticky="e")
            


    def play_song_admin(self, lagu):
        """Memutar lagu, mengisi Queue dengan lagu Linked List berikutnya."""
        # 1. Reset Queue dan History
        self.logic.playback_queue = PlaybackQueue() 
        self.logic.history = PlaybackHistory()

        # 2. Isi Queue dengan lagu-lagu berikutnya (Multi Linked List)
        # Karena LL adalah tunggal, kita hanya mengisi dengan node next_lagu
        curr = lagu.next_lagu 
        while curr:
            self.logic.playback_queue.enqueue(curr)
            curr = curr.next_lagu
            
        # 3. Putar lagu yang dipilih
        self.logic.play_lagu(lagu)
        self.controller.playback_bar.update_playback_info()
        messagebox.showinfo("Memutar", f"Memutar: {lagu.judul} oleh {lagu.artis}. Queue diisi dari Library.")


    def cari_lagu(self):
        kunci = self.search_option.get()
        nilai = self.search_entry.get().strip()
        
        if not nilai:
            self.refresh_content(reset_search=True)
            return

        hasil = self.logic.library.cari_lagu(kunci, nilai)
        if not hasil:
            messagebox.showinfo("Pencarian", f"Tidak ditemukan lagu dengan '{kunci}' mengandung '{nilai}'.")
        
        self.refresh_content(search_results=hasil)

    def open_update_song(self):
        if self.selected_song_id:
            lagu = self.logic.library.cari_lagu_berdasarkan_id(self.selected_song_id)
            if lagu:
                # Pindah ke halaman Tambah/Update dengan mode update
                self.controller.show_frame("AdminAddUpdateSongPage", mode="update", lagu_node=lagu)
            else:
                 messagebox.showerror("Error", "Lagu tidak ditemukan di library.")

    def hapus_lagu(self):
        """Menghapus lagu yang dipilih."""
        if self.selected_song_id:
            response = messagebox.askyesno("Konfirmasi Hapus", f"Yakin ingin menghapus lagu ID: {self.selected_song_id}?\nAksi ini juga akan menghapus lagu dari semua playlist pengguna.")
            if response:
                if self.logic.library.hapus_lagu_dan_update_playlist(self.selected_song_id, self.logic.users):
                    self.logic._save_library()
                    self.logic._save_users()
                    messagebox.showinfo("Berhasil", "Lagu berhasil dihapus.")
                    self.refresh_content(reset_search=True)
                else:
                    messagebox.showerror("Gagal", "Gagal menghapus lagu. ID tidak valid.")
        else:
            messagebox.showwarning("Perhatian", "Pilih lagu yang akan dihapus.")

    def logout(self):
        self.logic.current_user = None
        self.controller.toggle_playback_bar(show=False)
        self.controller.show_frame("LoginPage")


# Halaman Tambah/Update Lagu Admin
class AdminAddUpdateSongPage(ctk.CTkFrame):
    def __init__(self, parent, controller, logic):
        super().__init__(parent)
        self.controller = controller
        self.logic = logic
        self.mode = "tambah" # 'tambah' atau 'update'
        self.lagu_node = None # Objek Lagu jika mode 'update'
        self.selected_file_path = "" # Path file audio
        
        self.grid_columnconfigure(0, weight=1)
        self.grid_rowconfigure(1, weight=1)

        ctk.CTkButton(self, text="< KEMBALI", command=self.kembali_ke_admin).grid(row=0, column=0, sticky="w", padx=10, pady=10)

        main_frame = ctk.CTkFrame(self)
        main_frame.grid(row=1, column=0, padx=20, pady=20, sticky="nsew")
        
        self.title_label = ctk.CTkLabel(main_frame, text="TAMBAH LAGU BARU", font=ctk.CTkFont(size=24, weight="bold"))
        self.title_label.pack(pady=(20, 10))

        # Form Input
        form_frame = ctk.CTkFrame(main_frame, fg_color="transparent")
        form_frame.pack(padx=20, pady=20)

        self.fields = {}
        labels = ["Judul", "Artis", "Genre", "Album", "Durasi (m:ss)"]
        for i, label in enumerate(labels):
            ctk.CTkLabel(form_frame, text=f"{label}:", width=120, anchor="w").grid(row=i, column=0, padx=5, pady=5)
            entry = ctk.CTkEntry(form_frame, width=250)
            entry.grid(row=i, column=1, padx=5, pady=5)
            self.fields[label] = entry
            
        # Input File Audio
        self.file_path_label = ctk.CTkLabel(form_frame, text="File Audio:", width=120, anchor="w")
        self.file_path_label.grid(row=len(labels), column=0, padx=5, pady=5)
        
        file_frame = ctk.CTkFrame(form_frame, fg_color="transparent")
        file_frame.grid(row=len(labels), column=1, padx=5, pady=5, sticky="ew")
        
        self.file_display_label = ctk.CTkLabel(file_frame, text="[Belum Ada File Dipilih]", width=150, anchor="w")
        self.file_display_label.pack(side="left", padx=(0, 5))
        
        ctk.CTkButton(file_frame, text="Pilih File", command=self.select_audio_file, width=80).pack(side="left")

        # Tombol Aksi
        self.button_frame = ctk.CTkFrame(main_frame, fg_color="transparent")
        self.button_frame.pack(pady=20)
        
        self.save_button = ctk.CTkButton(self.button_frame, text="TAMBAH LAGU", command=self.save_lagu)
        self.save_button.pack(side="left", padx=10)
        
        self.cancel_button = ctk.CTkButton(self.button_frame, text="BATAL / KEMBALI", command=self.kembali_ke_admin, fg_color="gray", hover_color="#646464")
        self.cancel_button.pack(side="right", padx=10)

    def refresh_content(self, mode="tambah", lagu_node=None):
        self.mode = mode
        self.lagu_node = lagu_node
        self.selected_file_path = ""
        
        # Reset Form
        for entry in self.fields.values():
            entry.delete(0, tk.END)
        self.file_display_label.configure(text="[Belum Ada File Dipilih]")

        if mode == "tambah":
            self.title_label.configure(text="TAMBAH LAGU BARU")
            self.save_button.configure(text="TAMBAH LAGU")
            
        elif mode == "update" and lagu_node:
            self.title_label.configure(text=f"UPDATE LAGU (ID: {lagu_node.id_lagu})")
            self.save_button.configure(text="SIMPAN PERUBAHAN")
            
            # Isi form dengan data yang sudah ada
            self.fields["Judul"].insert(0, lagu_node.judul)
            self.fields["Artis"].insert(0, lagu_node.artis)
            self.fields["Genre"].insert(0, lagu_node.genre)
            self.fields["Album"].insert(0, lagu_node.album)
            self.fields["Durasi (m:ss)"].insert(0, lagu_node.durasi)
            self.selected_file_path = lagu_node.file_audio
            self.file_display_label.configure(text=os.path.basename(lagu_node.file_audio))

    def select_audio_file(self):
        """Membuka dialog untuk memilih file audio (MP3/WAV)."""
        file_path = filedialog.askopenfilename(
            defaultextension=".mp3",
            filetypes=[("Audio Files", "*.mp3 *.wav"), ("All Files", "*.*")]
        )
        if file_path:
            self.selected_file_path = file_path
            self.file_display_label.configure(text=os.path.basename(file_path))

    def save_lagu(self):
        judul = self.fields["Judul"].get().strip()
        artis = self.fields["Artis"].get().strip()
        genre = self.fields["Genre"].get().strip()
        album = self.fields["Album"].get().strip()
        durasi = self.fields["Durasi (m:ss)"].get().strip()
        file_audio = self.selected_file_path

        if not all([judul, artis, genre, album, durasi]):
            messagebox.showerror("Gagal", "Semua informasi lagu harus diisi.")
            return

        if not file_audio or file_audio == "simulated_path.mp3":
            messagebox.showerror("Gagal", "Anda harus memilih file audio.")
            return

        if self.mode == "tambah":
            self.logic.library.tambah_lagu(judul, artis, genre, album, durasi, file_audio)
            messagebox.showinfo("Berhasil", "Lagu baru berhasil ditambahkan.")
            
        elif self.mode == "update" and self.lagu_node:
            # Update data di node lagu yang sudah ada
            self.lagu_node.judul = judul
            self.lagu_node.artis = artis
            self.lagu_node.genre = genre
            self.lagu_node.album = album
            self.lagu_node.durasi = durasi
            self.lagu_node.file_audio = file_audio # Update path jika file baru dipilih
            messagebox.showinfo("Berhasil", f"Lagu ID: {self.lagu_node.id_lagu} berhasil diupdate.")

        self.logic._save_library()
        self.kembali_ke_admin()

    def kembali_ke_admin(self):
        self.controller.show_frame("AdminDashboardPage", reset_search=True)


# Halaman Utama User
class UserDashboardPage(ctk.CTkFrame):
    def __init__(self, parent, controller, logic):
        super().__init__(parent)
        self.controller = controller
        self.logic = logic
        
        self.grid_rowconfigure(1, weight=1) # Konten utama
        self.grid_columnconfigure(0, weight=1)

        # Header User
        self.header_frame = ctk.CTkFrame(self, fg_color="#60A060")
        self.header_frame.grid(row=0, column=0, sticky="ew", padx=0, pady=0)
        self.greeting_label = ctk.CTkLabel(self.header_frame, text="Selamat datang User!", font=ctk.CTkFont(size=20, weight="bold"))
        self.greeting_label.pack(pady=10, padx=20, anchor="w")

        # Konten Utama
        self.content_frame = ctk.CTkFrame(self, fg_color="transparent")
        self.content_frame.grid(row=1, column=0, sticky="nsew", padx=20, pady=10)
        self.content_frame.grid_columnconfigure((0, 1, 2, 3), weight=1)
        self.content_frame.grid_rowconfigure((0, 1), weight=0)
        self.content_frame.grid_rowconfigure(2, weight=1)

        # Tombol Aksi Cepat
        self.menu_frame = ctk.CTkFrame(self.content_frame, fg_color="transparent")
        self.menu_frame.grid(row=1, column=0, columnspan=4, sticky="ew", pady=(10, 20))
        self.menu_frame.grid_columnconfigure((0, 1, 2, 3), weight=1)

        ctk.CTkButton(self.menu_frame, text="🔍 Pencarian Lagu", command=lambda: controller.show_frame("UserSearchPage")).grid(row=0, column=0, padx=10, sticky="ew")
        ctk.CTkButton(self.menu_frame, text="➕ Buat Playlist", command=self.buat_playlist).grid(row=0, column=1, padx=10, sticky="ew")
        ctk.CTkButton(self.menu_frame, text="🎧 Playlist Saya", command=lambda: controller.show_frame("UserPlaylistMenuPage")).grid(row=0, column=2, padx=10, sticky="ew")
        ctk.CTkButton(self.menu_frame, text="🖼️ Katalog Artis", command=self.show_artist_catalogue).grid(row=0, column=3, padx=10, sticky="ew")


        # Area Gambar Artis (Simulasi)
        ctk.CTkLabel(self.content_frame, text="ARTIS POPULER", font=ctk.CTkFont(size=18, weight="bold")).grid(row=2, column=0, columnspan=4, sticky="w", pady=(10, 5))
        self.artist_grid = ctk.CTkFrame(self.content_frame, fg_color="transparent")
        self.artist_grid.grid(row=3, column=0, columnspan=4, sticky="nsew")
        self.content_frame.grid_rowconfigure(3, weight=1)

        # Tombol Keluar (di luar content_frame)
        self.logout_button = ctk.CTkButton(self, text="🚪 Keluar", command=self.logout)
        self.logout_button.grid(row=3, column=0, sticky="se", padx=20, pady=20)

    def refresh_content(self):
        if self.logic.current_user:
            self.greeting_label.configure(text=f"Selamat datang {self.logic.current_user.username}!")
        else:
            self.greeting_label.configure(text="Selamat datang User!")
        
        # Tampilkan atau refresh Katalog Artis
        self.show_artist_catalogue()

    def show_artist_catalogue(self):
        # Bersihkan grid
        for widget in self.artist_grid.winfo_children():
            widget.destroy()

        all_lagu = self.logic.library.get_all_lagu()
        artists = sorted(list(set(l.artis for l in all_lagu)))
        
        # Batasi jumlah yang ditampilkan (misalnya 4 artis)
        display_artists = artists[:4]
        self.artist_grid.grid_columnconfigure(list(range(len(display_artists))), weight=1)

        for i, artist in enumerate(display_artists):
            artist_frame = ctk.CTkFrame(self.artist_grid)
            artist_frame.grid(row=0, column=i, padx=10, pady=10, sticky="nsew")
            
            # Simulasi Gambar
            ctk.CTkLabel(artist_frame, text=artist[0].upper(), font=ctk.CTkFont(size=40, weight="bold"), width=100, height=100, fg_color="gray").pack(pady=5)
            ctk.CTkLabel(artist_frame, text=artist, font=ctk.CTkFont(weight="bold")).pack(pady=5)
            ctk.CTkButton(artist_frame, text="Lihat Lagu", command=lambda a=artist: self.controller.show_frame("UserArtistListPage", artist_name=a)).pack(pady=10)


    def buat_playlist(self):
        """Menampilkan dialog untuk membuat playlist baru."""
        if not self.logic.current_user:
            messagebox.showerror("Error", "Anda harus login sebagai user untuk membuat playlist.")
            return

        dialog = ctk.CTkInputDialog(text="Masukkan NAMA Playlist baru:", title="Buat Playlist Baru")
        nama = dialog.get_input()
        
        if nama:
            nama = nama.strip()
            if not nama:
                messagebox.showerror("Gagal", "Nama Playlist tidak boleh kosong.")
                return
            if nama in self.logic.current_user.playlists:
                messagebox.showerror("Gagal", "Nama Playlist sudah ada!")
                return
            
            self.logic.current_user.playlists[nama] = PlaylistLinkedList()
            self.logic._save_users()
            messagebox.showinfo("Berhasil", f"Playlist '{nama}' berhasil dibuat!")

    def logout(self):
        self.logic.current_user = None
        self.controller.toggle_playback_bar(show=False)
        self.controller.show_frame("UserSignupPage")


# Halaman List Lagu Artis (Queue Playback)
class UserArtistListPage(ctk.CTkFrame):
    def __init__(self, parent, controller, logic):
        super().__init__(parent)
        self.controller = controller
        self.logic = logic
        self.artist_name = None
        
        self.grid_rowconfigure(1, weight=1)
        self.grid_columnconfigure(0, weight=1)

        # Header dan Kembali
        header_frame = ctk.CTkFrame(self, fg_color="transparent")
        header_frame.grid(row=0, column=0, sticky="ew", padx=10, pady=10)
        header_frame.grid_columnconfigure(1, weight=1)

        ctk.CTkButton(header_frame, text="< KEMBALI", command=lambda: controller.show_frame("UserDashboardPage")).grid(row=0, column=0, sticky="w")
        self.artist_label = ctk.CTkLabel(header_frame, text="Lagu-Lagu Artis", font=ctk.CTkFont(size=20, weight="bold"))
        self.artist_label.grid(row=0, column=1, padx=20)


        # List Lagu (Scrollable)
        self.list_scroll_frame = ctk.CTkScrollableFrame(self)
        self.list_scroll_frame.grid(row=1, column=0, sticky="nsew", padx=10, pady=10)
        self.list_frame = ctk.CTkFrame(self.list_scroll_frame, fg_color="transparent")
        self.list_frame.pack(fill="x", expand=True)

        # Konfigurasi kolom Header
        self.list_frame.grid_columnconfigure(0, weight=0) # No
        self.list_frame.grid_columnconfigure(1, weight=3) # Judul
        self.list_frame.grid_columnconfigure(2, weight=1) # Genre
        self.list_frame.grid_columnconfigure(3, weight=1) # Album
        self.list_frame.grid_columnconfigure(4, weight=0) # Durasi
        self.list_frame.grid_columnconfigure(5, weight=0) # Aksi

    def refresh_content(self, artist_name):
        self.artist_name = artist_name
        self.artist_label.configure(text=f"Katalog Lagu: {artist_name}")

        for widget in self.list_frame.winfo_children():
            widget.destroy()

        # Header Tabel
        headers = ["No", "Judul", "Genre", "Album", "Durasi", "Aksi"]
        for i, text in enumerate(headers):
            ctk.CTkLabel(self.list_frame, text=text, font=ctk.CTkFont(weight="bold"), fg_color="#444444", height=30).grid(row=0, column=i, sticky="ew", padx=(1 if i > 0 else 0), pady=(0, 2))

        # Data Lagu
        lagu_list = self.logic.library.cari_lagu("artis", artist_name)
        
        for i, lagu in enumerate(lagu_list):
            row = i + 1
            row_frame = ctk.CTkFrame(self.list_frame, fg_color="transparent", border_width=0)
            row_frame.grid(row=row, column=0, columnspan=6, sticky="ew", pady=(0, 1))

            # FIX: Tentukan bobot kolom di dalam row_frame agar align dengan header (Perbaikan Tampilan Tabel)
            row_frame.grid_columnconfigure(0, weight=0) # No
            row_frame.grid_columnconfigure(1, weight=3) # Judul
            row_frame.grid_columnconfigure(2, weight=1) # Genre
            row_frame.grid_columnconfigure(3, weight=1) # Album
            row_frame.grid_columnconfigure(4, weight=0) # Durasi
            row_frame.grid_columnconfigure(5, weight=0) # Aksi
            
            # Kolom Data
            data_columns = [
                (f"{i+1}.", 0, tk.W),
                (lagu.judul, 1, tk.W),
                (lagu.genre, 2, tk.W),
                (lagu.album, 3, tk.W),
                (lagu.durasi, 4, tk.W)
            ]
            
            for text, col, anchor in data_columns:
                ctk.CTkLabel(row_frame, text=text, anchor=anchor, padx=5, pady=3).grid(row=0, column=col, sticky="ew")

            # Tombol Aksi: Play (Queue) dan Tambah ke Playlist
            action_frame = ctk.CTkFrame(row_frame, fg_color="transparent")
            action_frame.grid(row=0, column=5, sticky="e", padx=5)
            
            ctk.CTkButton(action_frame, text="▶️", width=30, command=lambda l=lagu: self.play_song_user_queue(l, lagu_list)).pack(side="left", padx=2)
            ctk.CTkButton(action_frame, text="➕", width=30, command=lambda l=lagu: self.add_to_playlist_dialog(l)).pack(side="left", padx=2)

    def play_song_user_queue(self, lagu_mulai, lagu_list):
        """Memutar lagu dan mengisi queue menggunakan tipe data Queue."""
        # 1. Reset Queue dan History
        self.logic.playback_queue = PlaybackQueue() 
        self.logic.history = PlaybackHistory()
        
        # 2. Isi Queue dengan lagu-lagu berikutnya (berdasarkan urutan di list)
        start_queue = False
        for lagu in lagu_list:
            if lagu.id_lagu == lagu_mulai.id_lagu:
                start_queue = True
                continue
            
            if start_queue:
                self.logic.playback_queue.enqueue(lagu)
                
        # 3. Putar lagu yang dipilih
        self.logic.play_lagu(lagu_mulai)
        self.controller.playback_bar.update_playback_info()
        messagebox.showinfo("Memutar", f"Memutar: {lagu_mulai.judul} oleh {lagu_mulai.artis}. Queue diisi.")

    def add_to_playlist_dialog(self, lagu):
        """Dialog untuk memilih playlist tujuan."""
        if not self.logic.current_user.playlists:
            messagebox.showwarning("Perhatian", "Anda belum memiliki Playlist. Buat Playlist baru terlebih dahulu.")
            return

        playlists = list(self.logic.current_user.playlists.keys())
        
        # Pop-up window
        top = ctk.CTkToplevel(self)
        top.title(f"Tambahkan '{lagu.judul}'")
        top.geometry("300x200")
        top.transient(self.controller)
        top.grab_set()

        ctk.CTkLabel(top, text=f"Pilih Playlist untuk: {lagu.judul}", font=ctk.CTkFont(weight="bold")).pack(pady=10)
        
        selected_playlist = tk.StringVar(value=playlists[0]) if playlists else None
        
        option_menu = ctk.CTkOptionMenu(top, values=playlists, variable=selected_playlist)
        option_menu.pack(pady=5)
        
        def save_to_playlist():
            nama_playlist = selected_playlist.get()
            playlist_ll = self.logic.current_user.playlists[nama_playlist]
            
            if playlist_ll.tambah_lagu(lagu):
                self.logic._save_users()
                messagebox.showinfo("Berhasil", f"Lagu berhasil ditambahkan ke '{nama_playlist}'.")
            else:
                messagebox.showwarning("Perhatian", f"Lagu sudah ada di '{nama_playlist}'.")
            
            top.destroy()

        ctk.CTkButton(top, text="Tambah", command=save_to_playlist).pack(pady=10)


# Halaman Pencarian Lagu (Mirip Artist List, tapi menampilkan semua hasil)
class UserSearchPage(ctk.CTkFrame):
    def __init__(self, parent, controller, logic):
        super().__init__(parent)
        self.controller = controller
        self.logic = logic
        
        self.grid_rowconfigure(2, weight=1)
        self.grid_columnconfigure(0, weight=1)

        # Header dan Kembali
        header_frame = ctk.CTkFrame(self, fg_color="transparent")
        header_frame.grid(row=0, column=0, sticky="ew", padx=10, pady=10)
        ctk.CTkButton(header_frame, text="< KEMBALI", command=lambda: controller.show_frame("UserDashboardPage")).pack(side="left", padx=5)
        ctk.CTkLabel(header_frame, text="PENCARIAN LAGU", font=ctk.CTkFont(size=20, weight="bold")).pack(side="left", padx=20)


        # Search Input Frame
        search_input_frame = ctk.CTkFrame(self)
        search_input_frame.grid(row=1, column=0, sticky="ew", padx=10, pady=5)
        search_input_frame.grid_columnconfigure(1, weight=1)
        
        self.search_entry = ctk.CTkEntry(search_input_frame, placeholder_text="Cari Judul, Artis, Genre, Album...", width=300)
        self.search_entry.grid(row=0, column=0, sticky="ew", padx=(10, 5), pady=10)
        self.search_option = ctk.CTkOptionMenu(search_input_frame, values=["judul", "artis", "genre", "album"], width=100)
        self.search_option.grid(row=0, column=1, padx=(5, 10), pady=10)
        ctk.CTkButton(search_input_frame, text="CARI", command=self.perform_search, width=80).grid(row=0, column=2, padx=(0, 10), pady=10)


        # List Lagu (Scrollable)
        self.list_scroll_frame = ctk.CTkScrollableFrame(self)
        self.list_scroll_frame.grid(row=2, column=0, sticky="nsew", padx=10, pady=10)
        self.list_frame = ctk.CTkFrame(self.list_scroll_frame, fg_color="transparent")
        self.list_frame.pack(fill="x", expand=True)

        # Konfigurasi kolom Header
        self.list_frame.grid_columnconfigure(0, weight=0) # No
        self.list_frame.grid_columnconfigure(1, weight=3) # Judul
        self.list_frame.grid_columnconfigure(2, weight=2) # Artis
        self.list_frame.grid_columnconfigure(3, weight=1) # Genre
        self.list_frame.grid_columnconfigure(4, weight=1) # Album
        self.list_frame.grid_columnconfigure(5, weight=0) # Durasi
        self.list_frame.grid_columnconfigure(6, weight=0) # Aksi
        
        # HAPUS PANGGILAN INI. FUNGSI refresh_content akan dipanggil oleh controller.show_frame
        # self.refresh_content(search_results=self.logic.library.get_all_lagu()) # Tampilkan semua lagu by default

    def perform_search(self):
        kunci = self.search_option.get()
        nilai = self.search_entry.get().strip()
        
        if not nilai:
            # Jika input kosong, tampilkan semua lagu (search_results=None)
            self.refresh_content()
            return

        hasil = self.logic.library.cari_lagu(kunci, nilai)
        if not hasil:
            messagebox.showinfo("Pencarian", f"Tidak ditemukan lagu dengan '{kunci}' mengandung '{nilai}'.")
        
        # Panggil refresh_content untuk menampilkan hasil, terlepas dari hasilnya (kosong/ada)
        self.refresh_content(search_results=hasil)

    # PERBAIKAN: Mengubah search_results menjadi parameter opsional
    def refresh_content(self, search_results=None):
        lagu_list = search_results
        if lagu_list is None:
            # Jika dipanggil tanpa argumen (dari menu), tampilkan semua lagu
            lagu_list = self.logic.library.get_all_lagu()

        for widget in self.list_frame.winfo_children():
            widget.destroy()

        # Header Tabel
        headers = ["No", "Judul", "Artis", "Genre", "Album", "Durasi", "Aksi"]
        for i, text in enumerate(headers):
            ctk.CTkLabel(self.list_frame, text=text, font=ctk.CTkFont(weight="bold"), fg_color="#444444", height=30).grid(row=0, column=i, sticky="ew", padx=(1 if i > 0 else 0), pady=(0, 2))

        # Data Lagu
        
        for i, lagu in enumerate(lagu_list):
            row = i + 1
            row_frame = ctk.CTkFrame(self.list_frame, fg_color="transparent", border_width=0)
            row_frame.grid(row=row, column=0, columnspan=7, sticky="ew", pady=(0, 1))

            # FIX: Tentukan bobot kolom di dalam row_frame agar align dengan header (Perbaikan Tampilan Tabel)
            row_frame.grid_columnconfigure(0, weight=0) # No
            row_frame.grid_columnconfigure(1, weight=3) # Judul
            row_frame.grid_columnconfigure(2, weight=2) # Artis
            row_frame.grid_columnconfigure(3, weight=1) # Genre
            row_frame.grid_columnconfigure(4, weight=1) # Album
            row_frame.grid_columnconfigure(5, weight=0) # Durasi
            row_frame.grid_columnconfigure(6, weight=0) # Aksi

            # Kolom Data
            data_columns = [
                (f"{i+1}.", 0, tk.W),
                (lagu.judul, 1, tk.W),
                (lagu.artis, 2, tk.W),
                (lagu.genre, 3, tk.W),
                (lagu.album, 4, tk.W),
                (lagu.durasi, 5, tk.W)
            ]
            
            for text, col, anchor in data_columns:
                ctk.CTkLabel(row_frame, text=text, anchor=anchor, padx=5, pady=3).grid(row=0, column=col, sticky="ew")

            # Tombol Aksi: Play (Queue) dan Tambah ke Playlist
            action_frame = ctk.CTkFrame(row_frame, fg_color="transparent")
            action_frame.grid(row=0, column=6, sticky="e", padx=5)
            
            ctk.CTkButton(action_frame, text="▶️", width=30, command=lambda l=lagu: self.play_song_user_queue(l, lagu_list)).pack(side="left", padx=2)
            # Menggunakan metode add_to_playlist_dialog dari UserArtistListPage
            ctk.CTkButton(action_frame, text="➕", width=30, command=lambda l=lagu: self.controller.get_frame("UserArtistListPage").add_to_playlist_dialog(l)).pack(side="left", padx=2)

    def play_song_user_queue(self, lagu_mulai, lagu_list):
        """Memutar lagu dan mengisi queue menggunakan tipe data Queue."""
        # Logika sama dengan di UserArtistListPage
        self.logic.playback_queue = PlaybackQueue() 
        self.logic.history = PlaybackHistory()
        
        start_queue = False
        for lagu in lagu_list:
            if lagu.id_lagu == lagu_mulai.id_lagu:
                start_queue = True
                continue
            
            if start_queue:
                self.logic.playback_queue.enqueue(lagu)
                
        self.logic.play_lagu(lagu_mulai)
        self.controller.playback_bar.update_playback_info()
        messagebox.showinfo("Memutar", f"Memutar: {lagu_mulai.judul} oleh {lagu_mulai.artis}. Queue diisi.")


# Halaman Menu Playlist User dan Detail Playlist
class UserPlaylistMenuPage(ctk.CTkFrame):
    def __init__(self, parent, controller, logic):
        super().__init__(parent)
        self.controller = controller
        self.logic = logic
        self.current_view = "menu" # 'menu' (daftar playlist) atau 'detail' (list lagu playlist)
        self.current_playlist_name = None
        
        self.grid_rowconfigure(1, weight=1)
        self.grid_columnconfigure(0, weight=1)
        
        # Header dan Kembali
        header_frame = ctk.CTkFrame(self, fg_color="transparent")
        header_frame.grid(row=0, column=0, sticky="ew", padx=10, pady=10)
        header_frame.grid_columnconfigure(1, weight=1)

        self.back_button = ctk.CTkButton(header_frame, text="< KEMBALI", command=self.go_back)
        self.back_button.grid(row=0, column=0, sticky="w")
        self.title_label = ctk.CTkLabel(header_frame, text="PLAYLIST SAYA", font=ctk.CTkFont(size=20, weight="bold"))
        self.title_label.grid(row=0, column=1, padx=20)

        # Konten utama (Scrollable Frame)
        self.content_scroll_frame = ctk.CTkScrollableFrame(self)
        self.content_scroll_frame.grid(row=1, column=0, sticky="nsew", padx=10, pady=10)
        self.content_frame = ctk.CTkFrame(self.content_scroll_frame, fg_color="transparent")
        self.content_frame.pack(fill="x", expand=True)

    def go_back(self):
        if self.current_view == "detail":
            self.refresh_content(view="menu")
        else:
            self.controller.show_frame("UserDashboardPage")

    def refresh_content(self, view="menu", playlist_name=None):
        """Memuat ulang tampilan menu/detail playlist."""
        if not self.logic.current_user:
            self.controller.show_frame("UserSignupPage")
            return
        
        self.current_view = view
        self.current_playlist_name = playlist_name

        for widget in self.content_frame.winfo_children():
            widget.destroy()
        
        self.content_frame.grid_columnconfigure(0, weight=1)

        if view == "menu":
            self.title_label.configure(text="PLAYLIST SAYA")
            self.display_playlist_menu()
            self.back_button.configure(command=lambda: self.controller.show_frame("UserDashboardPage"))
        elif view == "detail" and playlist_name:
            self.title_label.configure(text=f"Detail Playlist: {playlist_name}")
            self.display_playlist_detail(playlist_name)
            self.back_button.configure(command=lambda: self.refresh_content(view="menu"))
        else:
            self.display_playlist_menu()

    def display_playlist_menu(self):
        """Menampilkan daftar semua playlist."""
        playlists = self.logic.current_user.playlists
        
        if not playlists:
             ctk.CTkLabel(self.content_frame, text="Anda belum memiliki Playlist. Klik 'Buat Playlist' di Dashboard.", pady=20).pack()
             return

        ctk.CTkLabel(self.content_frame, text="Pilih Playlist:", font=ctk.CTkFont(weight="bold")).pack(pady=10, anchor="w")

        for name, playlist_ll in playlists.items():
            count = len(playlist_ll.get_lagu_list())
            playlist_button = ctk.CTkButton(self.content_frame, text=f"🎵 {name} ({count} lagu)", command=lambda n=name: self.refresh_content(view="detail", playlist_name=n))
            playlist_button.pack(fill="x", pady=5, padx=10)
            
    def display_playlist_detail(self, playlist_name):
        """Menampilkan list lagu di dalam playlist."""
        playlist_ll = self.logic.current_user.playlists.get(playlist_name)
        if not playlist_ll:
            messagebox.showerror("Error", "Playlist tidak ditemukan.")
            self.refresh_content(view="menu")
            return

        lagu_list = playlist_ll.get_lagu_list()
        
        # Konfigurasi kolom detail di content_frame (Outer container)
        self.content_frame.grid_columnconfigure(0, weight=0) # No
        self.content_frame.grid_columnconfigure(1, weight=3) # Judul
        self.content_frame.grid_columnconfigure(2, weight=2) # Artis
        self.content_frame.grid_columnconfigure(3, weight=1) # Genre
        self.content_frame.grid_columnconfigure(4, weight=0) # Aksi
        
        # Header Tabel
        headers = ["No", "Judul", "Artis", "Genre", "Aksi"]
        for i, text in enumerate(headers):
            ctk.CTkLabel(self.content_frame, text=text, font=ctk.CTkFont(weight="bold"), fg_color="#444444", height=30).grid(row=0, column=i, sticky="ew", padx=(1 if i > 0 else 0), pady=(0, 2))

        if not lagu_list:
             ctk.CTkLabel(self.content_frame, text="Playlist ini masih kosong.", pady=20).grid(row=1, column=0, columnspan=5)
             return
             
        for i, lagu in enumerate(lagu_list):
            row = i + 1
            row_frame = ctk.CTkFrame(self.content_frame, fg_color="transparent", border_width=0)
            row_frame.grid(row=row, column=0, columnspan=5, sticky="ew", pady=(0, 1))
            
            # FIX: Tentukan bobot kolom di dalam row_frame agar align dengan header (Perbaikan Tampilan Tabel)
            row_frame.grid_columnconfigure(0, weight=0) # No
            row_frame.grid_columnconfigure(1, weight=3) # Judul
            row_frame.grid_columnconfigure(2, weight=2) # Artis
            row_frame.grid_columnconfigure(3, weight=1) # Genre
            row_frame.grid_columnconfigure(4, weight=0 ) # Aksi
            
            # Kolom Data
            data_columns = [
                (f"{i+1}.", 0, tk.W),
                (lagu.judul, 1, tk.W),
                (lagu.artis, 2, tk.W),
                (lagu.genre, 3, tk.W)
            ]
            
            for text, col, anchor in data_columns:
                ctk.CTkLabel(row_frame, text=text, anchor=anchor, padx=5, pady=3).grid(row=0, column=col, sticky="ew")

            # Tombol Play & Hapus
            action_frame = ctk.CTkFrame(row_frame, fg_color="transparent")
            action_frame.grid(row=0, column=4, sticky="e", padx=5)
            
            ctk.CTkButton(action_frame, text="▶️", width=30, command=lambda l=lagu, p=playlist_ll: self.play_song_from_playlist(l, p)).pack(side="left", padx=2)
            ctk.CTkButton(action_frame, text="❌", width=30, fg_color="red", hover_color="#800000", command=lambda l_id=lagu.id_lagu, p_name=playlist_name: self.remove_song_from_playlist(l_id, p_name)).pack(side="left", padx=2)


    def play_song_from_playlist(self, lagu_mulai, playlist_ll):
        """Memutar lagu, mengisi queue menggunakan Linked List dengan fallback genre."""
        # 1. Reset Queue dan History
        self.logic.playback_queue = PlaybackQueue() 
        self.logic.history = PlaybackHistory()
        
        # 2. Isi Queue dengan lagu-lagu berikutnya (berdasarkan urutan di Linked List)
        curr = playlist_ll.head
        start_queue = False
        while curr:
            if curr.lagu.id_lagu == lagu_mulai.id_lagu:
                start_queue = True
                curr = curr.next
                continue
            
            if start_queue:
                self.logic.playback_queue.enqueue(curr.lagu)
            
            curr = curr.next
            
        # 3. Handle Fallback Genre (jika playlist habis)
        if self.logic.playback_queue.is_empty():
            similar_lagu = self.logic.library.find_similar_lagu_genre(lagu_mulai.genre, exclude_id=lagu_mulai.id_lagu)
            if similar_lagu:
                for lagu in similar_lagu:
                    self.logic.playback_queue.enqueue(lagu)
                messagebox.showwarning("Info Playlist", f"Lagu dalam playlist habis. Memuat lagu bergenre '{lagu_mulai.genre}' ke dalam queue.")
            
        # 4. Putar lagu yang dipilih
        self.logic.play_lagu(lagu_mulai)
        self.controller.playback_bar.update_playback_info()
        messagebox.showinfo("Memutar Playlist", f"Memutar: {lagu_mulai.judul} dari Playlist '{self.current_playlist_name}'. Queue diisi.")

    def remove_song_from_playlist(self, lagu_id, playlist_name):
        """Menghapus lagu dari playlist yang sedang dibuka."""
        response = messagebox.askyesno("Konfirmasi Hapus", f"Yakin ingin menghapus lagu ini dari Playlist '{playlist_name}'?")
        if response:
            playlist_ll = self.logic.current_user.playlists.get(playlist_name)
            if playlist_ll and playlist_ll.hapus_lagu_berdasarkan_id(lagu_id):
                self.logic._save_users()
                messagebox.showinfo("Berhasil", "Lagu berhasil dihapus dari playlist.")
                self.refresh_content(view="detail", playlist_name=playlist_name)
            else:
                messagebox.showerror("Gagal", "Gagal menghapus lagu.")


# ------------------------
# EKSEKUSI APLIKASI
# ------------------------
if __name__ == "__main__":
    # Inisialisasi Logika/Backend
    app_logic = SyfoplayAppLogic()
    
    # Inisialisasi GUI
    app_gui = SyfoplayGUI(app_logic)
    app_gui.mainloop()

c:\Users\USER\AppData\Local\Programs\Python\Python313\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.13.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
[INFO] Pygame Mixer berhasil diinisialisasi.
